__Hedge Calculations__

- 1. Defining hedge function
- 2. Defining hedge function with commission
- 3. Defining payout function
- 4. Defining payout function with commission
- 5. Defining profitable lay price boundary (given a some bp, the max lp needed for a profitable hedge bet)



In [1]:
import numpy as np

In [2]:
back_price = 2.5
back_size = 15
lay_price = 2
commission = 0.05
result = 'win'

# 1

In [3]:
def hedging(back_price, back_size, lay_price):
    '''
    Function applies hedging to identify lay_size
    Assumption : commission = 0%
    '''
    
    return round((back_price * back_size) / lay_price, 2)

In [4]:
lay_size = hedging(back_price, back_size, lay_price)
print(lay_size)

18.75


# 2

In [4]:
def hedging_comm(back_price, back_size, lay_price, commission):
    '''
    Function applies hedging to identify lay_size
    Assumption : commission = 0%
    '''
    back_net = (back_size * (back_price - 1)) * (1 - commission)
    return round((back_size + back_net) / lay_price, 2)

In [5]:
lay_size = hedging_comm(2, 100, 1.95, 0.05)
print(lay_size)

100.0


# 3

In [7]:
def payout(back_price, back_size, lay_price, lay_size, result):
    '''
    Function calculates payout of result based on positions and result.
    '''
    
    if result == 'win':
        back_payout = (back_price * back_size) - back_size
        lay_payout = - ((lay_price * lay_size) - lay_size)
        print(back_payout, lay_payout)
        
    else:
        back_payout = -back_size
        lay_payout = lay_size
        print(back_payout, lay_payout)
        
    return round(back_payout + lay_payout, 2)
    

In [8]:
payout(back_price, back_size, lay_price, lay_size, 'win')

22.5 -18.19


4.31

# 4

In [9]:
def payout_comm(back_price, back_size, lay_price, lay_size, result, commission):
    '''
    Function calculates payout of result based on positions and result.
    '''
    
    if result == 'win':
        back_payout = ((back_price * back_size) - back_size) * (1-commission)
        lay_payout = - ((lay_price * lay_size) - lay_size)
        print(back_payout, lay_payout)
        
    else:
        back_payout = -back_size
        lay_payout = (lay_size) * (1-commission)
        print(back_payout, lay_payout)
        
    return round(round(back_payout, 2) + round(lay_payout, 2), 2)

In [13]:
payout_comm(back_price==2.5, back_size==15, lay_price==, lay_size, 'win', 0.05)

0.0 -100.0


-100.0

# 5

In [1]:
def min_hedge_price(bp, c):
    '''
    Given some back price and commission, this returns the minimum the price for a profitable hedge bet
    '''
    
    decrease = (bp * c) - c
    lp = bp - decrease
    
    return lp

In [4]:
print(min_hedge_price(87, 0.05))

82.7


In [ ]:
^^^ need to test if this is the same now !

In [18]:
def derived_function(bp, bs, c, lp):
    return (((bp - 1) * bs * (1 - c)) + bs) / (lp - c)

In [19]:
derived_function(2, 1000, 0.05, 1.95)

1026.3157894736842

In [3]:
"""
Ed's functions start here
"""

def lay_stake(bp, bs, c, lp):
    return (((bp - 1) * bs * (1 - c)) + bs) / (lp - c)

def win_payout(bp, bs, c, lp):
    ls = lay_stake(bp, bs, c, lp)
    back_net = ((bp - 1) * bs * (1 - c))
    lay_net = - ls * (lp - 1)
    return back_net + lay_net

def loss_payout(bp, bs, c, lp):
    ls = lay_stake(bp, bs, c, lp)
    back_net = - bs
    lay_net = ls * (1 - c)
    return back_net + lay_net

def bet(bp, bs, c, lp):
    print(lay_stake(bp, bs, c, lp))
    print(win_payout(bp, bs, c, lp))
    print(str(loss_payout(bp, bs, c, lp)) + "\n")
    
bet(2.0, 1000.0, 0.05, 1.95)
bet(3.45, 245, 0.05, 1.0)
bet(2.6, 15000, 0.05, 2.4)

1026.3157894736842
-24.999999999999886
-25.000000000000114

858.1447368421052
570.2375
570.2375

16085.106382978724
280.8510638297885
280.85106382978665



In [4]:
def min_hedge_price(bp, c):
    '''
    Given some back price and commission, this returns the maximum lay price for a profitable hedge bet. The lower the lay price, the more profitable the bet.
    '''
    return (bp - 1) * (1 - c) ** 2 + 1

lp_max = min_hedge_price(2.0, 0.05)
print(str(lp_max) + "\n")

#0 profit bet
bet(2.0, 1000, 0.05, lp_max) 
#profitable bet, laying at lp_max - 0.01 = 1.890249999
bet(2.0, 1000, 0.05, lp_max - 0.01)
#even more profitable bets
bet(2.0, 1000, 0.05, lp_max - 0.03)
bet(2.0, 1000.0, 0.05, lp_max - 0.1)
bet(2.0, 1000.0, 0.05, lp_max - 0.2)

1.9024999999999999

1052.6315789473686
0.0
1.1368683772161603e-13

1058.3446404341928
5.427408412483032
5.427408412483146

1069.9588477366256
16.460905349794416
16.46090534979419

1112.6961483594866
57.061340941512185
57.0613409415123

1180.0302571860818
121.02874432677766
121.02874432677777



In [5]:
"""
taking random prices and determining whether or not to bet with a backing stake of 1,000
"""

from random import uniform
from random import normalvariate

bs = 1000
c = 0.05
total_return = 0

for i in range(1, 6):
    bp = uniform(0.0, 20.0)
    lp = max(normalvariate(bp, 5), 0)
    ls = lay_stake(bp, bs, c, lp)
    lp_max = min_hedge_price(bp, c)
    payout = win_payout(bp, bs, c, lp)
    
    if lp > lp_max:
        print("At back price of %f and lay price of %f, there is no profitable hedge. Betting £%d and laying £%d would give a guaranteed loss of -£%d. Lay price must be below %f" % (bp, lp, bs, ls, abs(payout), lp_max))
        print("Total return after %d bets is %d" % (i, total_return))
    elif 0.01 < lp < lp_max:
        total_return += payout
        print("At back price of %f and lay price of %f, there is a profitable hedge. Betting £%d and laying £%d gives a guaranteed profit of £%d." % (bp, lp, bs, ls, payout))
        print("Total return after %d bets is %d" % (i, total_return))

At back price of 17.605144 and lay price of 18.095293, there is no profitable hedge. Betting £1000 and laying £929 would give a guaranteed loss of -£116. Lay price must be below 15.986142
Total return after 1 bets is 0
At back price of 10.793659 and lay price of 5.264620, there is a profitable hedge. Betting £1000 and laying £1975 gives a guaranteed profit of £877.
Total return after 3 bets is 877
At back price of 4.805234 and lay price of 6.153178, there is no profitable hedge. Betting £1000 and laying £756 would give a guaranteed loss of -£281. Lay price must be below 4.434223
Total return after 4 bets is 877
At back price of 3.560971 and lay price of 8.339559, there is no profitable hedge. Betting £1000 and laying £414 would give a guaranteed loss of -£606. Lay price must be below 3.311276
Total return after 5 bets is 877


In [12]:
"""
Weighting the backing stake with a factor related to implied odds

tried w = (1 - 1 / (10 * exp(bp)))
      w = 1 - (bp - lp) / 100      #this will be a bad reflection of probability
      w = 1 - max((1 / lp - 1 / bp) / 10, 0)
"""

#from math import exp #if doing exp()

def weighted_lay_stake(bp, bs, c, lp):
    w = 1 - max((1 / lp - 1 / bp) / 10, 0)
    return ((((bp - 1) * bs * (1 - c)) + bs) / (lp - c) ) * w

def weighted_win_payout(bp, bs, c, lp):
    ls = weighted_lay_stake(bp, bs, c, lp)
    back_net = ((bp - 1) * bs * (1 - c))
    lay_net = - ls * (lp - 1)
    return back_net + lay_net

def weighted_loss_payout(bp, bs, c, lp):
    ls = weighted_lay_stake(bp, bs, c, lp)
    back_net = - bs
    lay_net = ls * (1 - c)
    return back_net + lay_net

def bet(bp, bs, c, lp):
    print(weighted_lay_stake(bp, bs, c, lp))
    print(weighted_win_payout(bp, bs, c, lp))
    print(str(weighted_loss_payout(bp, bs, c, lp)) + "\n")
    
bet(2.0, 1000.0, 0.05, min_hedge_price(2.0, 0.05))    
bet(2.0, 1000.0, 0.05, min_hedge_price(2.0, 0.05) - 0.03)
bet(2.0, 1000.0, 0.05, min_hedge_price(2.0, 0.05) - 0.1)
bet(2.0, 1000.0, 0.05, min_hedge_price(2.0, 0.05) - 0.2)

"""
vs in the non-weighted system:
1052.6315789473686
0.0
1.1368683772161603e-13

1069.9588477366256 #lay stake
16.460905349794416 #win side
16.46090534979419 #loss side

1112.6961483594866
57.061340941512185
57.0613409415123

1180.0302571860818
121.02874432677766
121.02874432677777
"""

1049.9342969776612
2.4342969776608925
-2.5624178712218963

1066.31613069827
19.639175965759705
13.000324163356368

1106.6002401957974
61.953307242872825
51.270228186007444

1169.7201543516367
128.2715915679754
111.2341466340547



'\nvs in the non-weighted system:\n1052.6315789473686\n0.0\n1.1368683772161603e-13\n\n1069.9588477366256 #lay stake\n16.460905349794416 #win side\n16.46090534979419 #loss side\n\n1112.6961483594866\n57.061340941512185\n57.0613409415123\n\n1180.0302571860818\n121.02874432677766\n121.02874432677777\n'